<a href="https://colab.research.google.com/github/jungwoo1208/AI_Study/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(128)
if device == 'cuda':
    torch.cuda.manual_seed_all(128)

In [2]:
learning_rate = 0.001
training_epochs = 100
batch_size = 100

In [3]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 480kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.46MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.46MB/s]


In [4]:
data_loader= torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [7]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=torch.nn.Linear(7*7*64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0), -1)
        out=self.fc(out)
        return out

In [8]:
model= CNN().to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
for epoch in range(training_epochs):
    avg_cost = 0

    for x, y in data_loader:
      x=x.to(device)
      y=y.to(device)

      optimizer.zero_grad()
      hypothesis = model(x)

      cost = criterion(hypothesis, y)
      cost.backward()
      optimizer.step()

      avg_cost += cost / len(data_loader)

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.223997593
[Epoch:    2] cost = 0.062595509
[Epoch:    3] cost = 0.0436661988
[Epoch:    4] cost = 0.0366139933
[Epoch:    5] cost = 0.029463144
[Epoch:    6] cost = 0.0247481726
[Epoch:    7] cost = 0.021132132
[Epoch:    8] cost = 0.0184578486
[Epoch:    9] cost = 0.0146780433
[Epoch:   10] cost = 0.0123129496
[Epoch:   11] cost = 0.011236826
[Epoch:   12] cost = 0.00924949162
[Epoch:   13] cost = 0.00737076066
[Epoch:   14] cost = 0.00704437587
[Epoch:   15] cost = 0.00715427659
[Epoch:   16] cost = 0.00470992923
[Epoch:   17] cost = 0.00451084413
[Epoch:   18] cost = 0.0048788595
[Epoch:   19] cost = 0.00479119457
[Epoch:   20] cost = 0.00434465287
[Epoch:   21] cost = 0.00342329848
[Epoch:   22] cost = 0.000768102997
[Epoch:   23] cost = 0.0046130768
[Epoch:   24] cost = 0.00350973057
[Epoch:   25] cost = 0.00411201874
[Epoch:   26] cost = 0.00246764696
[Epoch:   27] cost = 0.000993931782
[Epoch:   28] cost = 0.00181870849
[Epoch:   29] cost = 0.00356248929
[

In [11]:
with torch.no_grad():
  x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  y_test = mnist_test.test_labels.to(device)

  prediction = model(x_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9900999665260315


/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
